<a id='report-production'></a>
<p style="font-weight:bold;"> <span style="font-size: 36px"> Reports</span> </p>

For demonstration purposes we import here data for some *Group of Insurance Contract* (GIC) and *Group of Reinsurance Contract* (GRIC) - the import is triggered in the [Set up data and configuration](#set-up-data-and-configuration) section.
<br> The imported data set consists of cash flows, actuals, and parameters.
<br> Input files can be found in the **File** directory. You are invited to change them or upload your own or add new data in the [CloseImportTemplate](../Import/CloseImportTemplate) notebook. 
<br> For simplicity, we import similar transactional data for all GICs and GRICs. Each *Group of Contracts* produces different figures due to differences in parameters such as *Liability Type*, *Oci type* or *Premium allocation factor* to Contractual Service Margin.

Follow the instructions below for a guided interaction with the reports.

# Set up data and configuration

Choose to run the Reports notebook either with the set of Systemorph data in memory or with the data present in the Database: 
- #!eval-notebook "../Database/Configure" : connects to a physical Database
- #!eval-notebook "../Import/CloseImportTemplate" : uses the in-memory set up

We use here the in-memory set up.

In [0]:
#!eval-notebook "../Import/CloseImportTemplate"

In [0]:
Workspace.InitializeFrom(DataSource);
ifrs17.Reset(Workspace)

# Best Estimate

Present values of the [best-estimate](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.0/Report/ReportScopes#best-estimate) future cash flows are shown here in an Analysis of Change report.

The granularity of the reported figures can be modified by changing the Column Slices options.
For example one can add <code>"GroupOfContract"</code> to separate the contributions of the individual Group of Contracts.

Aggregated values are displayed when the data has a finer granularity than the one selected by the report slice options.

The Best Estimate scenario is selected by default. You can select other scenarios through the Scenario field. Possible options includes 'All' and 'Delta'. Select ColumnSlice Scenario if you chose one of these. 

In [0]:
var pv = ifrs17.PresentValues;
pv.ReportingNode = "CH";
pv.ReportingPeriod = (2020, 12);
pv.CurrencyType = CurrencyType.Contractual;
pv.ColumnSlices = new string[]{"GroupOfContract", "AmountType"};//"GroupOfContract", "AmountType", "Scenario", "ServicePeriod"
pv.Scenario = null; //"All";
pv.DataFilter = new [] {("GroupOfContract", "DT1.1")}; //new [] {("GroupOfContract", "DT1.2"),("LiabilityType", "LIC") };
(await pv.ToReportAsync)

# Risk Adjustment

Present values of the [risk adjustment](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.0/Report/ReportScopes#risk-adjustment) future cash flows are shown here.

The additional ColumnSlices are added to the view as the inner column. This can dicrease the readability of the report. For example, adding the slice by "GroupOfContract" leaves the lock-in and current rate contributions far apart and difficult to compare. The re-order of default slices with custom slices is achieved by esplicitly add the default slice among the custom slices. In our case, you can try entering both "GroupOfContract" and "EconomicBasis" separated by a comma in the ColumnSlices.

In [0]:
var ra = ifrs17.RiskAdjustments;
ra.ReportingNode = "CH";
ra.ReportingPeriod = (2020, 12);
ra.ColumnSlices = new string[]{"GroupOfContract"};//"GroupOfContract", "EconomicBasis"
ra.DataFilter = new [] {("GroupOfContract", "DT1.1")};//new [] {("GroupOfContract", "DT1.2")};
(await ra.ToReportAsync)

# Written Actuals

[Written Actuals](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.0/Report/ReportScopes#written-accrual-deferral) are shown here. 

In this case, the analysis of change view is replaced with a default slice by the **AmountTypes**. Only the amount type with non zero value are displayed. 
<br>Filters can be applied to reports in order to isolate a sub-set of the data. They are specified by the name of the dimension to filter and the system name of the desired value. For example, to investigate the contribution of a single Group of Contract the following filter can be applied: new [] {("GroupOfContract", "DT1.1")}.

In [0]:
var writtenActual = ifrs17.WrittenActuals;
writtenActual.ReportingNode = "CH";
writtenActual.ReportingPeriod = (2021, 3);
writtenActual.ColumnSlices = new string[]{"GroupOfContract"};//"GroupOfContract"
writtenActual.DataFilter =  new [] {("GroupOfContract", "DT1.1")}; //new [] {("GroupOfContract", "DT1.1")};
(await writtenActual.ToReportAsync)

## Advance, Overdue Actuals

Actuals payed in [Advance](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.0/Report/ReportScopes#written-accrual-deferral)
or [Overdue](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.0/Report/ReportScopes#written-accrual-deferral) are shown here together in a simplified Analysis of Change. 

Combining Filters and ColumnSlices facilitate the report analysis. For example, you can select "GroupOfContract" as column slices with a Filter on EstimateType "AA" to analyse the Advance Actuals for all Group of Contract. 

In [0]:
var accrualActual = ifrs17.AccrualActuals;
accrualActual.ReportingNode = "CH";
accrualActual.ReportingPeriod = (2021, 3);
accrualActual.ColumnSlices = new string[]{};//"GroupOfContract", "AmountType"
accrualActual.DataFilter = new [] {("GroupOfContract", "DT1.1")}; //new [] {("EstimateType", "AA")};
(await accrualActual.ToReportAsync)

## Deferrable Actuals

[Deferrable Actuals](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.0/Report/ReportScopes#written-accrual-deferral) are shown here. Amortization of the deferrable amount is computed using the Coverage Unit pattern.  

In [0]:
var deferrableActual = ifrs17.DeferralActuals;
deferrableActual.ReportingNode = "CH";
deferrableActual.ReportingPeriod = (2020, 12);
deferrableActual.ColumnSlices = new string[]{};//"GroupOfContract", "AmountType"
deferrableActual.DataFilter = new [] {("GroupOfContract", "DT1.1")};//new [] {("GroupOfContract", "DT1.1")};
(await deferrableActual.ToReportAsync)

# Fulfilment Cash flow

Present Value of the [Fulfilment Cash flow](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.0/Report/ReportScopes#fulfillment-cash-flows) are shown here. 
<br> The individual contributions from Best Estimate and Risk Adjustment can be visualized slicing by **EstimateType**.

Filters can be applied to report to isolate a sub-set of the data. For example you can Filter by a specific Group of Contract using its system name using : new [] {("GroupOfContract", "DT1.1")}.


In [0]:
var fulfillmentCashflows = ifrs17.FulfillmentCashflows;
fulfillmentCashflows.ReportingNode = "CH";
fulfillmentCashflows.ReportingPeriod = (2020, 12);
fulfillmentCashflows.ColumnSlices = new string[]{"GroupOfContract", "EstimateType"};//"EstimateType"
fulfillmentCashflows.DataFilter =  new [] {("GroupOfContract", "DT1.1")};
(await fulfillmentCashflows.ToReportAsync)

# Actuarial Experience Adjustment

A comparison between [Written Actual](#written-actual) and the Releases of the [Best Estimate](#present-value) is reported in the [Actuarial Experience Adjustment](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.0/Report/ReportScopes#experience-adjustment).

In [0]:
var experienceAdjustments = ifrs17.ExperienceAdjustments;
experienceAdjustments.ReportingNode = "CH";
experienceAdjustments.ReportingPeriod = (2020, 12);
experienceAdjustments.ColumnSlices = new string[]{"GroupOfContract","AmountType"};//"GroupOfContract", "AmountType"
experienceAdjustments.DataFilter = new [] {("GroupOfContract", "DT1.1")}; //new [] {("GroupOfContract", "DT1.1")};
(await experienceAdjustments.ToReportAsync)

# LRC Technical Margin

In the [Technical Margin](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.0/Report/ReportScopes#technical-margin) report we present a unified view on the figures that are allocated to either Contractual Service Margin or to Loss Component. 
<br> The Analysis of Change is expanded with few more steps such as **Experience Adjustment** and **Amortization**.

In [0]:
var technicalMargins = ifrs17.TechnicalMargins;
technicalMargins.ReportingNode = "CH";
technicalMargins.ReportingPeriod = (2020, 12);
technicalMargins.ColumnSlices = new string[]{"EstimateType"};//"GroupOfContract", "AmountType"
technicalMargins.DataFilter = new [] {("GroupOfContract", "DT1.1")};
(await technicalMargins.ToReportAsync)

# Contractual Service Margin / Loss Component / Loss Recovery Component

The Contractual Service Margin (CSM) / Loss Component (LC) / Loss Recovery Component (LR) [report](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.0/Report/ReportScopes#technical-margin) are here shown side by side as the allocation to profit or loss is done at each step of the Analysis of Change. 

A default slice by EstimateType - which distinguish between CSM, LC and LR contributions - is automatically enforced. 

In [0]:
var allocatedTechnicalMargins = ifrs17.AllocatedTechnicalMargins;
allocatedTechnicalMargins.ReportingNode = "CH";
allocatedTechnicalMargins.ReportingPeriod = (2020, 12);
allocatedTechnicalMargins.ColumnSlices = new string[]{};//"GroupOfContract", "EstimateType"
allocatedTechnicalMargins.DataFilter = new [] {("GroupOfContract", "DT1.1")}; //new [] {("GroupOfContract", "DT1.1")};
(await allocatedTechnicalMargins.ToReportAsync)

# LRC Actuarial

The [Actuarial Liability for Remaining Coverage](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.0/Report/ReportScopes#lrc-actuarial-actuarial-liability-for-remaining-coverage) report shows figures from Fulfilment Cash flow discounted with current yield curve, and the allocated techinical margin. 

In [0]:
var actuarialLrc = ifrs17.ActuarialLrc;
actuarialLrc.ReportingNode = "CH";
actuarialLrc.ReportingPeriod = (2020, 12);
actuarialLrc.ColumnSlices = new string[]{};//"GroupOfContract"
actuarialLrc.DataFilter = new [] {("GroupOfContract", "DT1.1")}; //new [] {("GroupOfContract", "DT1.1")};
(await actuarialLrc.ToReportAsync)

# LRC

The [Liability for Remaining Coverage](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.0/Report/ReportScopes#lrc-liability-for-remaining-coverage) report adds to the [Actuarial Liability for Remaining Coverage](#lrc-actuarial) the contribution of and accrual actuals. A simplified AoC Chain is used to allow comparison of the balance change between Actuals and Present Values. 

In [0]:
var lrc = ifrs17.Lrc;
lrc.ReportingNode = "CH";
lrc.ReportingPeriod = (2020, 12);
lrc.ColumnSlices = new string[]{};//"GroupOfContract",
lrc.DataFilter = new [] {("GroupOfContract", "DT1.1")}; //new [] {("GroupOfContract", "DT1.1")};
(await lrc.ToReportAsync)

# LIC Actuarial

The [Actuarial Liability of Incurred Claims](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.0/Report/ReportScopes#lic-actuarial-actuarial-liability-for-incurred-claims) report shows figures from Fulfilment Cash flow discounted with current yield curve. 

In [0]:
var actuarialLic = ifrs17.ActuarialLic;
actuarialLic.ReportingNode = "CH";
actuarialLic.ReportingPeriod = (2021, 3);
actuarialLic.ColumnSlices = new string[]{};//"GroupOfContract", "AmountType"
actuarialLic.DataFilter = null; //new [] {("GroupOfContract", "DT1.1")};
(await actuarialLic.ToReportAsync)

# LIC

The [Liability for Incurred Claims](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.0/Report/ReportScopes#lic-liability-for-incurred-claims) adds to the [Actuarial Liability for Incurred Claims](#lic-actuarial) the contribution of and accrual actuals.

In [0]:
var lic = ifrs17.Lic;
lic.ReportingNode = "CH";
lic.ReportingPeriod = (2021, 3);
lic.ColumnSlices = new string[]{};//"GroupOfContract", "AmountType"
lic.DataFilter = null; //new [] {("GroupOfContract", "DT1.1")};
(await lic.ToReportAsync)

# Financial Performance

The [Financial Performance](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.0/Report/ReportScopes#ifrs-17-financial-performance) report discloses the Change in Estimate of the IFRS 17 balance sheet items ([LRC](#lrc) and [LIC](#lic)) and the relevant incurred cash flows (Premiums, Claims, Expenses, etc...) for the given period.

Use the expand and collapse buttons in the report rows to change the granularity of the figures displayed.

In [0]:
var financialPerformance = ifrs17.FinancialPerformance;
financialPerformance.ReportingNode = "CH";
financialPerformance.ReportingPeriod = (2020, 12);
financialPerformance.ColumnSlices = new string[]{"GroupOfContract"};//"GroupOfContract"
financialPerformance.DataFilter = null;//new [] {("GroupOfContract", "DTR1.1")}; //new [] {("GroupOfContract", "DT1.1")};
(await financialPerformance.ToReportAsync) with {GroupDefaultExpanded = 3}

In [0]:
await Report.ForDataCube((ir+ise)
//.Filter(x => x.GroupOfContract == "DTR1.1")
)
    .WithQuerySource(Workspace)
    .SliceRowsBy(new [] {"VariableType", "EstimateType"})
    .SliceColumnsBy(new [] {"Currency", "GroupOfContract"}.Concat(financialPerformance.ColumnSlices).ToArray())
    .ReportGridOptions(headerColumnWidth: 500, groupDefaultExpanded: 3)
    .ExecuteAsync()

In [0]:
public interface InsuranceRevenue: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<InsuranceRevenue>(s => s.WithApplicability<InsuranceRevenueNotApplicable>(x => x.Identity.Id.IsReinsurance || x.Identity.Id.LiabilityType == LiabilityTypes.LIC));

    // PAA Premiums
    private IDataCube<ReportVariable> WrittenCashflow => GetScope<WrittenAndAccruals>(Identity).Written.Filter(("VariableType", "CF"));
    private IDataCube<ReportVariable> AdvanceWriteOff => GetScope<WrittenAndAccruals>(Identity).Advance.Filter(("VariableType", "WO"));
    private IDataCube<ReportVariable> OverdueWriteOff => GetScope<WrittenAndAccruals>(Identity).Overdue.Filter(("VariableType", "WO"));
    private IDataCube<ReportVariable> EffectiveActuals => WrittenCashflow -1 * (AdvanceWriteOff + OverdueWriteOff);   
    
    private IDataCube<ReportVariable> PaaPremiums => Identity.Id switch {
            { ValuationApproach: ValuationApproaches.PAA } => EffectiveActuals
                .Where(x => GetStorage().GetHierarchy<AmountType>().Ancestors(x.AmountType, includeSelf: true).Any(x => x.SystemName == AmountTypes.PR))
                .SelectToDataCube(v => v with { Novelty = Novelties.C, VariableType = "IR11" }),
            _ => Enumerable.Empty<ReportVariable>().ToArray().ToDataCube()
    };

    // Experience Adjustment On Premiums
    private IDataCube<ReportVariable> NotPaaActualPremiums => Identity.Id switch {
        { ValuationApproach: ValuationApproaches.PAA } => Enumerable.Empty<ReportVariable>().ToArray().ToDataCube(),
        _ => EffectiveActuals
            .Where(x => GetStorage().GetHierarchy<AmountType>().Ancestors(x.AmountType, includeSelf: true).Any(x => x.SystemName == AmountTypes.PR))
            .SelectToDataCube(v => v with { Novelty = Novelties.C }) 
    };

    private IDataCube<ReportVariable> NotPaaBestEstimatePremiums => Identity.Id switch {
        { ValuationApproach: ValuationApproaches.PAA } => Enumerable.Empty<ReportVariable>().ToArray().ToDataCube(),
        _ => GetScope<BestEstimate>(Identity).BestEstimate
            .Filter(("VariableType", "CF"), ("LiabilityType", "LRC"))
            .Where(x => GetStorage().GetHierarchy<AmountType>().Ancestors(x.AmountType, includeSelf: true).Any(x => x.SystemName == AmountTypes.PR))
            .AggregateOver(nameof(Novelty))
            .SelectToDataCube(rv => rv with { Novelty = Novelties.C })
    };

    private IDataCube<ReportVariable> WrittenPremiumsToCsm => GetScope<FxData>((Identity.Id, Identity.CurrencyType, EstimateTypes.APA)).FxData;
    private IDataCube<ReportVariable> BestEstimatePremiumsToCsm => GetScope<FxData>((Identity.Id, Identity.CurrencyType, EstimateTypes.BEPA)).FxData;

    private IDataCube<ReportVariable> ExperienceAdjustmentOnPremiums => (
            (NotPaaActualPremiums - WrittenPremiumsToCsm).AggregateOver(nameof(EstimateType)).SelectToDataCube(rv => rv with { EstimateType = EstimateTypes.A }) -
            (NotPaaBestEstimatePremiums - BestEstimatePremiumsToCsm).AggregateOver(nameof(EstimateType)).SelectToDataCube(rv => rv with { EstimateType = EstimateTypes.BE })
        )
        .AggregateOver(nameof(Novelty), nameof(VariableType))
        .SelectToDataCube(v => v with { Novelty = Novelties.C, VariableType = "IR12" });        

    // Expected Best Estimate cash flow out Release
    private IDataCube<ReportVariable> CfOut => -1 * GetScope<BestEstimate>(Identity).BestEstimate
        .Filter(("VariableType", "CF"), ("LiabilityType", "LRC"))
        .AggregateOver(nameof(Novelty))
        .SelectToDataCube(rv => rv with { Novelty = Novelties.C });
        
    private IDataCube<ReportVariable> ExpectedClaims => CfOut // --> Exclude NA Expenses
        .Where(x => GetStorage().GetHierarchy<AmountType>().Ancestors(x.AmountType, includeSelf: true).Any(x => x.SystemName == AmountTypes.CL))
        .SelectToDataCube(v => v with { VariableType = "IR13" });
    private IDataCube<ReportVariable> ExpectedClaimsInvestmentComponent => -1 * CfOut
        .Where(x => GetStorage().GetHierarchy<AmountType>().Ancestors(x.AmountType, includeSelf: true).Any(x => x.SystemName == AmountTypes.ICO))
        .SelectToDataCube(v => v with { VariableType = "IR2" });
    //private IDataCube<ReportVariable> ExpectedClaimsInvestmentComponentIse => -1 * ExpectedClaimsInvestmentComponentToIr;

    private IDataCube<ReportVariable> ExpectedExpenses => CfOut
        .Where(x => GetStorage().GetHierarchy<AmountType>().Ancestors(x.AmountType, includeSelf: true).Any(x => x.SystemName == AmountTypes.AE))
        .SelectToDataCube(v => v with { VariableType = "IR13" });

    private IDataCube<ReportVariable> ExpectedCommissions => CfOut
        .Where(x => GetStorage().GetHierarchy<AmountType>().Ancestors(x.AmountType, includeSelf: true).Any(x => x.SystemName == AmountTypes.AC))
        .SelectToDataCube(v => v with { VariableType = "IR13" });

    // RA Release
    private IDataCube<ReportVariable> RaRelease => -1 * GetScope<RiskAdjustment>(Identity).RiskAdjustment
        .Filter(("VariableType", "CF"), ("LiabilityType", "LRC"))
        .AggregateOver(nameof(Novelty))
        .SelectToDataCube(rv => rv with { Novelty = Novelties.C, VariableType = "IR13" });

    // CSM Release (Amortization)
    private IDataCube<ReportVariable> CsmAmortization => -1 * GetScope<Csm>(Identity).Csm
        .Filter(("VariableType", AocTypes.AM))
        .SelectToDataCube(v => v with { VariableType = "IR13" });
   
    // Loss Component Release (Amortization)
    private IDataCube<ReportVariable> LossComponentAmortization => GetScope<Lc>(Identity).Lc
        .Filter(("VariableType", AocTypes.AM))
        .SelectToDataCube(v => v with { VariableType = "IR13" });

    // Acquistion Expenses Release (Amortization)
    private IDataCube<ReportVariable> AcquistionExpensesAmortization => GetScope<Deferrals>(Identity)
        .Filter(("VariableType", AocTypes.AM))
        .SelectToDataCube(v => v with { VariableType = "IR13" });

    // InsuranceRevenue   
    IDataCube<ReportVariable> InsuranceRevenue => PaaPremiums + ExperienceAdjustmentOnPremiums + RaRelease + CsmAmortization + LossComponentAmortization + ExpectedClaims + ExpectedClaimsInvestmentComponent + ExpectedExpenses + ExpectedCommissions + AcquistionExpensesAmortization;

}

public interface InsuranceRevenueNotApplicable : InsuranceRevenue {
    IDataCube<ReportVariable> InsuranceRevenue.InsuranceRevenue => Enumerable.Empty<ReportVariable>().ToArray().ToDataCube(); 
}

In [0]:
public interface InsuranceServiceExpense: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<InsuranceServiceExpense>(s => s.WithApplicability<InsuranceServiceExpenseReinsurance>(x => x.Identity.Id.IsReinsurance));

    // Actuals cash flow out Release
    private IDataCube<ReportVariable> WrittenCashflow => GetScope<WrittenAndAccruals>(Identity).Written.Filter(("VariableType", "CF"));
    private IDataCube<ReportVariable> AdvanceWriteOff => GetScope<WrittenAndAccruals>(Identity).Advance.Filter(("VariableType", "WO"));
    private IDataCube<ReportVariable> OverdueWriteOff => GetScope<WrittenAndAccruals>(Identity).Overdue.Filter(("VariableType", "WO"));
    protected IDataCube<ReportVariable> EffectiveActuals => WrittenCashflow -1 * (AdvanceWriteOff + OverdueWriteOff);   
    
    private IDataCube<ReportVariable> ActualClaims => EffectiveActuals
        .Where(x => GetStorage().GetHierarchy<AmountType>().Ancestors(x.AmountType, includeSelf: true).Any(x => x.SystemName == AmountTypes.CL))
        .SelectToDataCube(v => v with { VariableType = "ISE2" });
    private IDataCube<ReportVariable> ActualClaimsInvestmentComponent => -1 * EffectiveActuals
        .Where(x => GetStorage().GetHierarchy<AmountType>().Ancestors(x.AmountType, includeSelf: true).Any(x => x.SystemName == AmountTypes.ICO))
        .SelectToDataCube(v => v with { VariableType = "ISE5" });

    private IDataCube<ReportVariable> ActualExpenses => EffectiveActuals
        .Where(x => GetStorage().GetHierarchy<AmountType>().Ancestors(x.AmountType, includeSelf: true).Any(x => x.SystemName == AmountTypes.AE))
        .SelectToDataCube(v => v with { VariableType = "ISE3" });

    private IDataCube<ReportVariable> ActualCommissions => EffectiveActuals
        .Where(x => GetStorage().GetHierarchy<AmountType>().Ancestors(x.AmountType, includeSelf: true).Any(x => x.SystemName == AmountTypes.AC))
        .SelectToDataCube(v => v with { VariableType = "ISE4" });

    // Acquistion Expenses Release (Amortization)
    private IDataCube<ReportVariable> AcquistionExpensesAmortization => -1 * GetScope<Deferrals>(Identity)
        .Filter(("VariableType", AocTypes.AM))
        .SelectToDataCube(v => v with { VariableType = "ISE6" });

    // Loss Component
    private IDataCube<ReportVariable> Lc => GetScope<Lc>(Identity).Lc.Filter(("VariableType", "!BOP"),("VariableType", "!EOP")) +
                                            GetScope<Lc>(Identity).Lc.Filter(("VariableType", AocTypes.BOP),("Novelty", Novelties.N));
    
    private IDataCube<ReportVariable> LcAmortization => -1 * Lc.Filter(("VariableType", AocTypes.AM)).SelectToDataCube(v => v with { VariableType = "ISE9" });
    
    private IDataCube<ReportVariable> LcNonFinancialChanges => -1 * Lc
        .Filter(("VariableType", "!AM"), ("VariableType", "!IA"), ("VariableType", "!YCU"), ("VariableType", "!CRU"), ("VariableType", "!FX"))
        .AggregateOver(nameof(Novelty), nameof(VariableType))
        .SelectToDataCube(v => v with { Novelty = Novelties.C, VariableType = "ISE11" });
    
    // private IDataCube<ReportVariable> Fx => -1 * Lc.Filter(("VariableType", AocTypes.FX))
    //     .AggregateOver(nameof(Novelty))
    //     .SelectToDataCube(v => v with { VariableType = "IFIE3" });

    // private IDataCube<ReportVariable> FinancialChanges => -1 * (Lc.Filter(("VariableType", AocTypes.IA)) + Lc.Filter(("VariableType", AocTypes.YCU)) + Lc.Filter(("VariableType", AocTypes.CRU)) )
    //     .AggregateOver(nameof(Novelty), nameof(VariableType))
    //     .SelectToDataCube(v => v with { Novelty = Novelties.C, VariableType = "IFIE1" });

    // Change in LIC
    private IDataCube<ReportVariable> FcfDeltas => (GetScope<Fcf>(Identity).Fcf.Filter(("VariableType", "!BOP"),("VariableType", "!EOP")) +
                                                   GetScope<Fcf>(Identity).Fcf.Filter(("VariableType", AocTypes.BOP),("Novelty", Novelties.N)))
                                                   .Filter(("LiabilityType", "LIC"));  // TODO, extract the LIC to a dedicated scope (whole thing, actually)
    
    private IDataCube<ReportVariable> NonFinancialFcfDeltas => FcfDeltas
        .Filter(("VariableType", "!IA"), ("VariableType", "!YCU"), ("VariableType", "!CRU"), ("VariableType", "!FX"));

    private IDataCube<ReportVariable> FpNonFinancialLic => -1 * NonFinancialFcfDeltas
        .AggregateOver(nameof(Novelty), nameof(VariableType))
        .SelectToDataCube(rv => rv with { Novelty = Novelties.C, VariableType = "ISE12" });

    // Reinsurance
    IDataCube<ReportVariable> Reinsurance => Enumerable.Empty<ReportVariable>().ToArray().ToDataCube(); 

    // Insurance Service Expense   
    IDataCube<ReportVariable> InsuranceServiceExpense => ActualClaims + ActualClaimsInvestmentComponent + ActualExpenses + ActualCommissions + AcquistionExpensesAmortization + LcAmortization + LcNonFinancialChanges + FpNonFinancialLic + Reinsurance;
}

public interface InsuranceServiceExpenseReinsurance : InsuranceServiceExpense {
    // Expected Best Estimate cash flow out Release
    private IDataCube<ReportVariable> CfOut => -1 * GetScope<BestEstimate>(Identity).BestEstimate
        .Filter(("VariableType", "CF"), ("LiabilityType", "LRC"))
        .AggregateOver(nameof(Novelty))
        .SelectToDataCube(rv => rv with { Novelty = Novelties.C });
        
    private IDataCube<ReportVariable> ExpectedClaims => CfOut // --> Exclude NA Expenses
        .Where(x => GetStorage().GetHierarchy<AmountType>().Ancestors(x.AmountType, includeSelf: true).Any(x => x.SystemName == AmountTypes.CL))
        .SelectToDataCube(v => v with { VariableType = "ISE22" });
    private IDataCube<ReportVariable> ExpectedClaimsInvestmentComponent => -1 * CfOut
        .Where(x => GetStorage().GetHierarchy<AmountType>().Ancestors(x.AmountType, includeSelf: true).Any(x => x.SystemName == AmountTypes.ICO))
        .SelectToDataCube(v => v with { VariableType = "ISE23" });
    //private IDataCube<ReportVariable> ExpectedClaimsInvestmentComponentIse => -1 * ExpectedClaimsInvestmentComponentToIr;

    private IDataCube<ReportVariable> ExpectedExpenses => CfOut
        .Where(x => GetStorage().GetHierarchy<AmountType>().Ancestors(x.AmountType, includeSelf: true).Any(x => x.SystemName == AmountTypes.AE))
        .SelectToDataCube(v => v with { VariableType = "ISE22" });

    private IDataCube<ReportVariable> ExpectedCommissions => CfOut
        .Where(x => GetStorage().GetHierarchy<AmountType>().Ancestors(x.AmountType, includeSelf: true).Any(x => x.SystemName == AmountTypes.AC))
        .SelectToDataCube(v => v with { VariableType = "ISE22" });

    // RA Release
    private IDataCube<ReportVariable> RaRelease => -1 * GetScope<RiskAdjustment>(Identity).RiskAdjustment
        .Filter(("VariableType", "CF"), ("LiabilityType", "LRC"))
        .AggregateOver(nameof(Novelty))
        .SelectToDataCube(rv => rv with { Novelty = Novelties.C, VariableType = "ISE22" });

    // CSM Release (Amortization)
    private IDataCube<ReportVariable> CsmAmortization => -1 * GetScope<Csm>(Identity).Csm
        .Filter(("VariableType", AocTypes.AM))
        .SelectToDataCube(v => v with { VariableType = "ISE22" });
   
    // Loss Recovery Component (Amortization)
    private IDataCube<ReportVariable> LoReCoAmortization => GetScope<Loreco>(Identity).Loreco
        .Filter(("VariableType", AocTypes.AM))
        .SelectToDataCube(v => v with { VariableType = "ISE22" });

    // Experience Adjustment On Premiums
    private IDataCube<ReportVariable> ReinsuranceActualPremiums => Identity.Id switch {
        { ValuationApproach: ValuationApproaches.PAA } => Enumerable.Empty<ReportVariable>().ToArray().ToDataCube(),
        _ => EffectiveActuals
            .Where(x => GetStorage().GetHierarchy<AmountType>().Ancestors(x.AmountType, includeSelf: true).Any(x => x.SystemName == AmountTypes.PR))
            .SelectToDataCube(v => v with { Novelty = Novelties.C }) 
    };

    private IDataCube<ReportVariable> ReinsuranceBestEstimatePremiums => Identity.Id switch {
        { ValuationApproach: ValuationApproaches.PAA } => Enumerable.Empty<ReportVariable>().ToArray().ToDataCube(),
        _ => GetScope<BestEstimate>(Identity).BestEstimate
            .Filter(("VariableType", "CF"), ("LiabilityType", "LRC"))
            .Where(x => GetStorage().GetHierarchy<AmountType>().Ancestors(x.AmountType, includeSelf: true).Any(x => x.SystemName == AmountTypes.PR))
            .AggregateOver(nameof(Novelty))
            .SelectToDataCube(rv => rv with { Novelty = Novelties.C })
    };

    private IDataCube<ReportVariable> ExperienceAdjustmentOnPremiums => (
            (ReinsuranceActualPremiums).AggregateOver(nameof(EstimateType)).SelectToDataCube(rv => rv with { EstimateType = EstimateTypes.A }) -
            (ReinsuranceBestEstimatePremiums).AggregateOver(nameof(EstimateType)).SelectToDataCube(rv => rv with { EstimateType = EstimateTypes.BE })
        )
        .AggregateOver(nameof(Novelty), nameof(VariableType))
        .SelectToDataCube(v => v with { Novelty = Novelties.C, VariableType = "ISE21" });

    // Reinsurance
    IDataCube<ReportVariable> InsuranceServiceExpense.Reinsurance => ExpectedClaims + ExpectedClaimsInvestmentComponent + ExpectedExpenses + ExpectedCommissions + RaRelease + CsmAmortization + LoReCoAmortization + ExperienceAdjustmentOnPremiums;
}


In [0]:
var reportStorage = new ReportStorage(Workspace, Report, Export);
await reportStorage.InitializeReportIndependentCacheAsync();
var mostRecentPartition = (await Workspace.Query<PartitionByReportingNodeAndPeriod>().Where(x => x.Scenario == null).OrderBy(x => x.Year).ThenBy(x => x.Month).ToArrayAsync()).Last();
var reportingNodeRoot = (await Workspace.Query<ReportingNode>().Where(x => x.Parent == null).ToArrayAsync()).First().SystemName;

((int Year, int Month) Period, string ReportingNode, string Scenario, CurrencyType CurrencyType) Args = ((2020,12), "CH", null,CurrencyType.Contractual);

await reportStorage.InitializeAsync(Args.Period, Args.ReportingNode, Args.Scenario, Args.CurrencyType);
var identities = reportStorage.GetIdentities(Args.Period, Args.ReportingNode, Args.Scenario, Args.CurrencyType);

var universe = Scopes.ForSingleton().WithStorage(reportStorage).ToScope<IUniverse>();

var ir = universe.GetScopes<InsuranceRevenue>(identities).Aggregate().InsuranceRevenue;
var ise = universe.GetScopes<InsuranceServiceExpense>(identities).Aggregate().InsuranceServiceExpense;